### 3.17. (Tallant una barra d'acer)
Tenim una gran barra d'acer de longitud $n$, i volem tallar-la en trossos per a destinar-los a diferents usos. Cada tros de barra d'acer, en funció de la seva llargària, té un cost al mercat. Un tros de llargària $i$, amb $i \in \mathbb{Z}^+$ i $1 \leq i \leq n$, val $\mathbf{p_i}$ euros. Observeu que les llargàries dels trossos són unitats enteres. Per exemple, una figura de llargaria 4 té 8 maneres de dividir la barra. Sobre cadascun dels trossos s'indica el preu que se n'obté ($\mathbf{p_1 = 1}$, $\mathbf{p_2 = 5}$, $\mathbf{p_3 = 8}$, $\mathbf{p_4 = 9}$). La solució òptima és tallar la barra en dos trossos de longitud 2  que dóna un guany de 10€.

$(a)$ De quantes formes diferents es pot tallar una barra de llargària $n$?Raoneu la resposta.

$(b)$ Dissenyeu un algorisme, el més eficient que pugueu, per a decidir com tallar una barra d'acer de longitud $n$ en trossos, de manera que es maximitzi el guany total que se n'obté. El vostre algorisme ha de dir quants talls s'han de fer en total, i a on.

*Nota:* Assumirem que fer un tall no indueix cap cost afegeix. Observeu que no es demana cap requisit sobre el nombre de talls a fer; podeu fer qualsevol nombre de talls entre $0$ i $n-1$.

#### Apartat a:
Per una barra de longitud $n$ en tenim $n-1$ punts de tall diferents. Per cada punt de tall tenim la decisió de si volem ``tallar`` o ``no tallar`` la barra d'acer. Per tant, el nombre de formes diferents que en tenim per una barra de llargària $n$ es de:

$$nº formes = \mathbf{2^{n-1}}$$

#### Apartat b:
##### Caracterització del problema
Per resoldre aquest exercici necesitem saber en cada moment, amb la forma que tenim actualment de la barra, quin es el preu que ens pagaran per ella. Per tant, l'objectiu es dividir la barra en totes les seves formes posibles, memoritzar el preu i després quedarnos amb la descomposició que te el màxim preu. Per fer la memoïtzació faré ús d'un array DP amb $n$ posicions que guardará quin es el preu màxim per cada barra de longitud desde 1 fins a longitud n. Començarem desde el cas base on la barra es de longitud ``1`` i per tant el seu preu es ``p(1)`` i terminarem en el cas en que la longitud de la barra es ``n`` que es el cas que volem solucionar.

Llavors, per saber la longitud d'una barra de longitud i em de tenir en compte els preus de totes les longituds anterior i el preu que ens donen per la barra sense tallar. La solució del cas recursiu queda així:
$$DP[i] = \max_{1 \le j \le i} (p(j) + DP[i - j])$$

##### Algorisme topdown

In [1]:
def cortar_barra_topdown(precios, n, memo=None):
    if memo is None:
        memo = {}
    if n == 0:
        return 0
    if n in memo:
        return memo[n]
    
    max_val = float('-inf')
    for i in range(1, n + 1):
        max_val = max(max_val, precios[i - 1] + cortar_barra_topdown(precios, n - i, memo))
    
    memo[n] = max_val
    return max_val

##### Algorisme amb reconstrucció topdown

In [2]:
def cortar_barra_topdown_sol(precios, n, memo=None, cortes_memo=None):
    if memo is None:
        memo = {}
        cortes_memo = {}
    if n == 0:
        return 0, []
    if n in memo:
        return memo[n], cortes_memo[n]
    
    max_val = float('-inf')
    mejor_corte = []
    
    for i in range(1, n + 1):
        valor, cortes = cortar_barra_topdown_sol(precios, n - i, memo, cortes_memo)
        valor_total = precios[i - 1] + valor
        if valor_total > max_val:
            max_val = valor_total
            mejor_corte = [i] + cortes
    
    memo[n] = max_val
    cortes_memo[n] = mejor_corte
    return max_val, mejor_corte

##### Algorisme bottomup

In [3]:
def cortar_barra_bottomup(precios, n):
    dp = [0] * (n + 1)
    
    for i in range(1, n + 1):
        max_val = float('-inf')
        for j in range(1, i + 1):
            max_val = max(max_val, precios[j - 1] + dp[i - j])
        dp[i] = max_val
    
    return dp[n]

##### Algorisme amb reconstrucció bottomup

In [4]:
def cortar_barra_bottomup_sol(precios, n):
    dp = [0] * (n + 1)
    cortes = [0] * (n + 1)
    
    for i in range(1, n + 1):
        max_val = float('-inf')
        mejor_corte = 0
        for j in range(1, i + 1):
            if precios[j - 1] + dp[i - j] > max_val:
                max_val = precios[j - 1] + dp[i - j]
                mejor_corte = j
        dp[i] = max_val
        cortes[i] = mejor_corte
    
    # reconstruir la solución óptima
    res = []
    longitud = n
    while longitud > 0:
        res.append(cortes[longitud])
        longitud -= cortes[longitud]
    
    return dp[n], res

Exemple d'us:

In [5]:
precios = [1, 5, 8, 9]  # p1=1, p2=5, p3=8, p4=9
n = 4

print("Preu máxim Topdown: " + str(cortar_barra_topdown(precios, n)))               # 10
print("Particions Topdown: " + str(cortar_barra_topdown_sol(precios, n)))           # (10, [2, 2])

print("Preu máxim Bottomup: " + str(cortar_barra_bottomup(precios, n)))              # 10
print("Preu máxim Bottomup: " + str(cortar_barra_bottomup_sol(precios, n)))          # (10, [2, 2])

Preu máxim Topdown: 10
Particions Topdown: (10, [2, 2])
Preu máxim Bottomup: 10
Preu máxim Bottomup: (10, [2, 2])


##### Cost de cada solució
El cost de la solució es en tots dos casos:
- Cost temporal: $O(n^2)$ en tot dos casos.
- Cost en memoria: $O(n)$ en la solució ``Bottomup`` i $O(n^2)$ en la solució ``Topdown``.

##### Correctessa:
L'algorisme es correcte, ja que té una subestructura optima i la recursivitat definida sempre ens dona el preu màxim que es pot obtenir per una barra de llargaria ``n``.